In [ ]:
import os

# Third-party
from astropy.table import Table
import astropy.coordinates as coord
from astropy.constants import G as _G
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.special import loggamma
from gala.units import UnitSystem

import pystan
from stderr_helper import suppress_stdout_stderr

Make some fake data

Tracer density:
$$
\nu(r) = \nu_0 \, r^{-\gamma} \\
\int \nu(r) \, r^2 \, {\rm d}r
$$

Background potential is Keplerian
$$
v_c(r) = \sqrt{\frac{G M}{r}}
$$

I'm going to work in some crazy-ass units:

In [ ]:
units = UnitSystem(u.mpc, u.yr, u.Msun, u.rad)
G = _G.decompose(units).value

In [ ]:
n_data = 512
Mbh = 4E6
gamma = 3.5
a = 2. # mpc
b = 1000. # mpc = 1 pc

In [ ]:
true_r, true_v = np.load('rv.npy')

In [ ]:
n_data = 32

r_err = np.full(n_data, 0.01)
r = np.random.normal(true_r[:n_data], r_err)

v_err = np.full(n_data, 0.01)
v = np.random.normal(true_v[:n_data], v_err)

In [ ]:
data_dict = dict(r=r, r_err=r_err,
                 v=v, v_err=v_err,
                 N=n_data)

In [ ]:
sm = pystan.StanModel('simple_model.stan')

In [ ]:
n_chains = 1
# init_dict = [dict(true_r=r,
#                   true_v=v,
#                   Mbh=true_Mbh,
# #                   phi0=50000.,
# #                   gamma=true_gamma
#                  )
#              for _ in range(n_chains)]

# HACK:
n_data = 128
data_dict = dict(true_r=true_r[:n_data], 
                 true_v=true_v[:n_data],
                 N=n_data)

init_dict = [dict(Mbh=Mbh,
#                   phi0=50000.,
#                   gamma=true_gamma
                 )
             for _ in range(n_chains)]

In [ ]:
fit = sm.sampling(data=data_dict, algorithm='HMC', iter=32, init=init_dict, 
                  chains=n_chains, n_jobs=1)

In [ ]:
with suppress_stdout_stderr():
    fit = sm.sampling(data=data_dict, algorithm='HMC', iter=8192, init=init_dict, 
                      chains=n_chains, n_jobs=1)

In [ ]:
_ = fit.traceplot('Mbh')

In [ ]:
plot_pars = ['Mbh']# 'gamma']
samples = fit.extract(plot_pars)

plt.figure()
plt.hist(samples['Mbh'], color='#666666') #, bins=np.logspace(4, 6, 32))
plt.axvline(true_Mbh, color='r')
# plt.xscale('log')

# plt.figure()
# plt.hist(samples['a_g'], color='#666666', bins=np.linspace(1, 30, 16))
# plt.axvline(true_alpha/true_gamma, color='r')

# plt.figure()
# plt.hist(samples['gamma'], color='#666666', bins=np.linspace(0, 1, 12))
# plt.axvline(true_gamma, color='r')